In [6]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import matplotlib.pyplot as plt
import functools as func
import contextily as ctx
import rasterio

from matplotlib.tri import Triangulation, LinearTriInterpolator
from rasterstats import point_query
from shapely.geometry import shape
from rasterio.plot import show

In [7]:
#read in spatial data, project to NAD83/Massachusetts Mainland
zcta = gpd.read_file(str(os.getcwd()) + r'/Data_Directory/vector/Census/tl_2010_25_zcta510/tl_2010_25_zcta510.shp').to_crs('epsg:26986')
colleges= gpd.read_file(str(os.getcwd()) + r'/Data_Directory/vector/MassGIS/COLLEGES_PT/COLLEGES_PT.shp').to_crs('epsg:26986')
MPO_bound = gpd.read_file(str(os.getcwd()) + r'/Data_Directory/vector/MassDOT/MPO_Boundaries/MPO_Boundaries.shp').to_crs('epsg:26986')

DriverError: '/Users/pjgomez/Fletcher/Boston_Site_Selection_UEP239/Data_Directory/vector/Census/tl_2010_25_zcta510/tl_2010_25_zcta510.shp' not recognized as a supported file format.

'/Users/pjgomez/Fletcher/Boston_Site_Selection_UEP239'

In [ ]:
print ("Zip Code Tabulation Area")
zcta.plot()
zcta.head(3)

In [ ]:
#wrangle tabular data

def get_column(path,col,rename_col):
    '''
    Isolates one column of interest from a csv and its corresponding GEO_ID,
    changes name from code to descriptive name
    '''
    df = pd.read_csv(str(os.getcwd()) + path) #convert csv to datafame
    
    df= df.filter(items= ['NAME',col])#filter column of interest
    
    df = df.rename(columns={col: rename_col}) #change column name

    df= df.drop(df.index[0]) #rid of second row
    
    return df


In [ ]:

#create dictionary in the following format: key= dataframe, value = list with two elements- code and descriptive name
d= {r'/Data_Directory/tabular/ACSST5Y2019.S2501/ACSST5Y2019.S2501_data_with_overlays.csv':['S2501_C01_010E', 'hh_married'],
    r'/Data_Directory/tabular/ACSST5Y2019.S1401/ACSST5Y2019.S1401_data_with_overlays_2021-05-03T211502.csv':['S1401_C03_003E','kids_in_ps'],
    r'/Data_Directory/tabular/ACSST5Y2019.S2503/ACSST5Y2019.S2503_data_with_overlays.csv':['S2503_C04_011E','pct_making_100K_150k'],
    r'/Data_Directory/tabular/ACSST5Y2019.S1501/ACSST5Y2019.S1501_data_with_overlays.csv':['S1501_C01_012E','over_25_with_bach_degree']}

#iterate through dictionary, add new dataframes to empty list
dfs= []
for key in d:
    print(key, '-> [', d[key][0], "|", d[key][1], "]")
    new_df = get_column(key, d[key][0], d[key][1])
    dfs.append(new_df)

#merge list of dataframes 
merged = func.reduce(lambda left, right: pd.merge(left, right, on=['NAME'], how='outer'), dfs)

#change numeric columns to numeric datatypes
cols=[i for i in merged.columns if i != "NAME"]
for c in cols:
    merged[c]=pd.to_numeric(merged[c], errors='coerce')

merged[['ZCTA','NAME']] = merged.NAME.str.split(expand=True)

#drop extra column
merged= merged.drop(['ZCTA'], axis = 1) 


In [ ]:
#join zcta
zcta_join = zcta.merge(merged, left_on='ZCTA5CE10', right_on = 'NAME')
zcta_join.shape

In [ ]:
zcta_join.plot(legend=True,column='pct_making_100K_150k', cmap = 'OrRd',legend_kwds={'label':'% of households making between $100-150K'}).set_title('Income')
zcta_join.plot(legend=True,column='hh_married', cmap = 'OrRd',legend_kwds={'label':'# of households'}).set_title('Married Households')
zcta_join.plot(legend=True,column='kids_in_ps', cmap = 'OrRd',legend_kwds={'label':'# of kids'}).set_title('Kids in Public Schools')
zcta_join.plot(legend=True,column='over_25_with_bach_degree', cmap = 'OrRd',legend_kwds={'label':'% with a BA or more'}).set_title('Educated households')


In [ ]:
plt.hist(zcta_join['hh_married'], bins=20)
plt.show() 

plt.hist(zcta_join['kids_in_ps'], bins=20)
plt.show() 

plt.hist(zcta_join['pct_making_100K_150k'], bins=20)
plt.show() 

plt.hist(zcta_join['over_25_with_bach_degree'], bins=20)

plt.show() 

In [ ]:
#create dictionary of ZCTA's top half thresholds

stats= zcta_join[cols].describe()[5:6]
top50pct= stats.values

top50pct_d= {}
for i in range(len(cols)):
    top50pct_d[cols[i]]= top50pct[0][i]

top50pct_d

In [ ]:
#select ZCTA's that meet the top half thresholds

conditional= []

for key in top50pct_d:
    clause = str(key) + " > " + str(top50pct_d[key]) + " &"
    conditional.append(clause)

criteria= str(conditional).replace(',','').replace("'","").replace("[","")

zcta_w_criteria= zcta_join.query(criteria[:-3])

print('Total ZCTAs: {0}\nZCTAs with demographic criteria: {1}'.format(len(zcta_join),len(zcta_w_criteria)))


In [ ]:
zcta_w_criteria.plot()

In [ ]:
#select colleges of interest, i.e., none that are 4 year insitutions (high potential for rowdiness)

colleges_no4yr= colleges[(colleges.NCES_TYPE != '4-year, Public') & (colleges.NCES_TYPE !='4-year, Private not-for-profit')]

#add new column to ZCTA's showing TRUE if a college resides in that ZCTA
inp, res = colleges_no4yr.sindex.query_bulk(zcta_w_criteria.geometry, predicate='intersects')
zcta_w_criteria['college_intersect'] = np.isin(np.arange(0, len(zcta_w_criteria)), inp)
zcta_w_criteria['GEOID10'].groupby(zcta_w_criteria['college_intersect']).agg(['nunique'])


In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

zcta_w_criteria.plot(column='college_intersect',
                categorical=True,
                legend=True,
                cmap= 'prism',
                ax=ax)

colleges_no4yr.plot(ax = ax,marker = '.',markersize=50, color = "grey", zorder= 2)
ctx.add_basemap(ax, url=ctx.providers.Stamen.TonerLite)
ax.set_axis_off()

In [ ]:
zcta_w_criteria = zcta_w_criteria[zcta_w_criteria.college_intersect == False]

#clip to Boston
bos = MPO_bound[MPO_bound.MPO == 'Boston Region'].copy()

zcta_clip = gpd.clip(zcta_w_criteria, bos)
zcta_clip.plot()

In [ ]:
#read in EPA air quality data from 2021
epa_raw = pd.read_csv(str(os.getcwd()) + r'/Data_Directory/tabular/EPA_PM25_2021.csv')

#convert to geospatial data, project
epa = gpd.GeoDataFrame(epa_raw, geometry=gpd.points_from_xy(epa_raw.SITE_LONGITUDE, epa_raw.SITE_LATITUDE))
epa = epa.set_crs('epsg:4326')
epa_prj= epa.to_crs('epsg:26986')

#make readable column name
epa_prj = epa_prj.rename(columns={'Daily Mean PM2.5 Concentration':'DMC_PM25'})
epa_prj.plot()

In [ ]:
#the following cells perform an interpolation method  from Saul Montoya at Hatari Labs. See readme for info.

#create numpy array of EPA data
totalPointsArray = np.zeros([epa_prj.shape[0],3])

#iteration over the geopandas dataframe
for index, point in epa_prj.iterrows():
    pointArray = np.array([point.geometry.coords.xy[0][0],point.geometry.coords.xy[1][0],point['DMC_PM25']])
    totalPointsArray[index] = pointArray
totalPointsArray


In [ ]:

#triangulation function
triFn = Triangulation(totalPointsArray[:,0],totalPointsArray[:,1])
#linear triangule interpolator funtion
linTriFn = LinearTriInterpolator(triFn,totalPointsArray[:,2])

#define raster resolution in (m)
rasterRes = 1000

xCoords = np.arange(totalPointsArray[:,0].min(), totalPointsArray[:,0].max()+rasterRes, rasterRes)
yCoords = np.arange(totalPointsArray[:,1].min(), totalPointsArray[:,1].max()+rasterRes, rasterRes)
zCoords = np.zeros([yCoords.shape[0],xCoords.shape[0]])

#loop among each cell in the raster extension
for indexX, x in np.ndenumerate(xCoords):
    for indexY, y in np.ndenumerate(yCoords):
        tempZ = linTriFn(x,y)
        #filtering masked values
        if tempZ == tempZ:
            zCoords[indexY,indexX]=tempZ
        else:
            zCoords[indexY,indexX]=np.NaN

#preliminary representation of the interpolated values
plt.imshow(zCoords)

In [ ]:
from rasterio.transform import Affine
transform = Affine.translation(xCoords[0] - rasterRes/2, yCoords[0] - rasterRes/2) * Affine.scale(rasterRes, rasterRes)
transform


In [ ]:
#definition, register and close of interpolated raster
full_ras_name = str(os.getcwd()+r'/Data_Directory/raster/epainterp.tif')
triInterpRaster = rasterio.open(full_ras_name),
                                'w',
                                driver='GTiff',
                                height=zCoords.shape[0],
                                width=zCoords.shape[1],
                                count=1,
                                dtype=zCoords.dtype,
                                crs={'init': 'epsg:26986'},
                                transform=transform)
triInterpRaster.write(zCoords,1)
triInterpRaster.close()

In [ ]:
ras = rasterio.open(full_ras_name)

fig, ax = plt.subplots(figsize=(15, 15))
rasterio.plot.show(ras, ax=ax)
zcta_clip.plot(ax=ax, facecolor='none', edgecolor='r').invert_yaxis()

#end of interpolation

In [ ]:
#change geometry to points to extract interpolated air quality 
zcta_points = zcta_w_criteria.copy()

zcta_points['geometry'] = zcta_points['geometry'].centroid

#extract air quality data from each point
zcta_w_airq= point_query(zcta_points, full_ras_name, geojson_out=True)

In [ ]:
#parse out geojson output to create a geodataframe
for d in zcta_w_airq:
    d['geometry'] = shape(d['geometry'])

zcta_pts_final = gpd.GeoDataFrame(zcta_w_airq).set_geometry('geometry')

#move air quality and GEOID values into new columns with nan values by parsing the 'properties' values from the JSON
zcta_pts_final['airqual']= np.nan
zcta_pts_final['GEOID']= np.nan


counter = 0
for i in zcta_pts_final.iterrows():
    i = counter
    zcta_pts_final['airqual'][i]=zcta_pts_final['properties'][i]['value']
    zcta_pts_final['GEOID'][i]=zcta_pts_final['properties'][i]['GEOID10']
    counter+=1

#rid of nan values
zcta_pts_final.dropna()

#find lowest 25% values
stats= zcta_pts_final['airqual'].describe()[4:5]
zcta_pts_final= zcta_pts_final[zcta_pts_final.airqual < stats[-1]]

#clean point layer
zcta_pts_final['GEOID']= zcta_pts_final['GEOID'].astype(np.int64) #rid of '.0'
zcta_pts_final['GEOID']= zcta_pts_final['GEOID'].astype(str) #convert to string
zcta_pts_final= zcta_pts_final.drop(['properties','bbox'], axis = 1) #rid extra columns

as_df= pd.DataFrame(zcta_pts_final)

#join back to zcta polygons, check resulting size of the new dataframe
zcta_merg = as_df.merge(zcta_clip, how= 'inner', left_on='GEOID', right_on = 'GEOID10')
zcta_final= zcta_merg.set_geometry('geometry_y')
zcta_final.shape

In [ ]:
#clean slivers from clip output
'''
WARNING: this is not reproducable.
9000 was chosen here because they were extreme outliers in the area colum
'''
zcta_final['area']=zcta_final.area
zcta_final= zcta_final[zcta_final.area > 9000]
zcta_final.shape
zcta_final

In [ ]:
#plot ideal ZCTAs on map
fig, ax = plt.subplots(figsize=(15, 15))
zcta_final.plot(ax=ax, facecolor='none', edgecolor='r', linewidth=5)
ctx.add_basemap(ax, url=ctx.providers.OpenStreetMap.Mapnik)
ax.set_axis_off()